In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Cubic spline interpolation task - given a set of known data points $x$ and function values at these points ($y = f(x)$), find a way to approximate function values in between (on a grid) of known points using the global piece-wise (spline) method with each piece interploated using 3-rd degree polynomials. The problem corresponds to finding correct coefficients for each polynomial $S_{3,i}(x)$ and undetermined parameters (function derivative values at points).

$S_3(x) \in C^{(2)}[a, b]$ - smoothness class<br>
$S_3(x) = \bigcup\limits_{i=0}^{n-1}S_{3,i}(x)$, defined at $x \in [a, b]$ and $i = \overline{0, n-1}$,
and combined from the union of 3-rd order polynomials $S_{3, i}(x)$ <br>
(1) $S_{3, i}(x) = \sum\limits_{k=0}^{3}a_{k,i}(x-x_i)^k = a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$, $x \in [x_i, x_{i+1}]$

## Assumption

**Let's assume that we use cubic spline according to first algorithm described in "Численные методы в примерах и задачах" by V.I.Kireev and A.V.Panteleev, 3-rd edition, 2008, page 188.** (Also was discussed during the lab 4)

This gives us differential orders for $p1$ and $p2$ as

$p_1 = \{0, 2\}$, $p_2 = \{1\}$

$p1$ and $p2$ are non-intersecting sets of derivative orders

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

- **General form for cubic spline**<br>
$\delta S_{3,i}^{(p_1)}(x_j) = S_{3,i}^{(p_1)}(x)|_{x=x_j} - f^{(p_1)}(x)|_{x=x_j} = 0, \ j=i,i+1$
<br>

- **Functional compatibility conditions ($p_1=0$)**<br>
(2) $\delta S_{3,i}(x_j) = S_{3,i}(x_j) - f(x_j) = 0, \ j=i,i+1$ (2 conditions: 1 for each $j$)
<br>

- **Differential compatibility conditions ($p_1=2$)**<br>
(3) $\delta S_{3,i}^{''}(x_j) = S_{3,i}^{''}(x_j) - f^{''}(x_j) = 0, \ j=i,i+1$ (2 conditions: 1 for each $j$)

### 3. Formulate stitching conditions [mathematical formula]

- **General form for cubic spline** <br>
$S_{3,i-1}^{(p_2)}(x)|_{x=x_i} = S_{3,i}^{(p_2)}(x)|_{x=x_i},\ i=\overline{1,n-1}$
<br>

- **Form for $p_1 = 1$** <br>
(4) $S_{3,i-1}^{'}(x)|_{x=x_i} = S_{3,i}^{'}(x)|_{x=x_i},\ i=\overline{1,n-1}$ 

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Required smoothness: $S_{3}(x) \in C^{(2)}[a, b]$, $C^{2}$ - class of functions which have continuous first and second derivatives.

Functional compatibility conditions help us to solve interpolation task (add equations) and make the whole function continuous.

Differential compatibility conditions (for $p = 2$) provide us with required smoothness ($C^{2}$) for all points in each piece of spline polynomial, except stitching points.

Stitching conditions help us to connect unknown part of parameters with already known parameters (such as express $f^{''}$ through $f$) and help to avoid absence of derivative in stitching point in order not to get such type of curve (picture for quadratic spline, but idea is the same)

![title](ex.png)

All these conditions help us to achieve required smoothness for $S_{3, i}(x)$

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

Let's use formulas (1), (2) and (3).

Compute derivatives for (1)
(1) $S_{3, i}(x) = a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$ <br>
$S_{3, i}^{'}(x) = a_{1,i} + 2a_{2,i}(x-x_i) + 3a_{3,i}(x-x_i)^2$ <br>
$S_{3, i}^{''}(x) = 2a_{2,i} + 6a_{3,i}(x-x_i)$

Plug (1) in (2) and (3) and receive system of linear equations, from which we will find coefficients of i-th polynomial.

$\begin{cases}
\delta S_{3,i}(x_i) = S_{3,i}(x_i) - f(x_i) = 0 \\
\delta S_{3,i}(x_{i+1}) = S_{3,i}(x_{i+1}) - f(x_{i+1}) = 0 \\
\delta S_{3,i}^{''}(x_i) = S_{3,i}^{''}(x_i) - f^{''}(x_i) = 0 \\
\delta S_{3,i}^{''}(x_{i+1}) = S_{3,i}^{''}(x_{i+1}) - f^{''}(x_{i+1}) = 0 \\
\end{cases}$

$\Downarrow$

$\begin{cases}
a_{0,i} - f_i = 0 \\
a_{0,i} + a_{1,i}(x_{i+1}-x_i) + a_{2,i}(x_{i+1}-x_i)^2 + a_{3,i}(x_{i+1}-x_i)^3 - f_{i+1} = 0 \\
2a_{2,i} - f_i^{''} = 0 \\
2a_{2,i} + 6a_{3,i}(x_{i+1} - x_i) - f_{i+1}^{''} = 0 \\
\end{cases}$

$\Downarrow$

Let <br>
$m_i = f_i^{''}$ <br>
$h_{i+1} = x_{i+1} - x_i$ <br>
$\Delta f_i = f_{i+1} - f_i$ <br>
$\Delta m_i = m_{i+1} - m_i$ <br>

$\begin{cases}
a_{0,i} = f_i  \\
a_{1,i} = \cfrac{1}{h_{i+1}}\Delta f_i - \cfrac{h_{i+1}}{2}m_i - \cfrac{h_{i+1}}{6}\Delta m_i \\
a_{2,i} = \cfrac{m_i}{2} \\
a_{3,i} = \cfrac{1}{6h_{i+1}}\Delta m_i \\
\end{cases}$

After finding coefficients, we obtain

$S_{3, i}(x) = f_i + 
[\cfrac{1}{h_{i+1}}\Delta f_i - \cfrac{h_{i+1}}{2}m_i - \cfrac{h_{i+1}}{6}\Delta m_i](x-x_i) + 
\cfrac{m_i}{2}(x-x_i)^2 + 
\cfrac{1}{6h_{i+1}}\Delta m_i(x-x_i)^3$

And now, we want to find undetermined parameters $m_i\ (i = \overline{0,n})$ <br>
This can be done using stitching conditions (4): <br>
$S_{3,i-1}^{'}(x)|_{x=x_i} = S_{3,i}^{'}(x)|_{x=x_i},\ i=\overline{1,n-1}$ <br>
$S_{3,i-1}$ piece for $x \in [x_{i-1}, x_i]$ <br>
$S_{3,i}$ piece for $x \in [x_{i}, x_{i+1}]$ <br>

$S_{3,i-1}^{'}(x)|_{x=x_i} = \cfrac{1}{h_{i}}\Delta f_{i-1} - \cfrac{h_{i}}{2}m_{i-1} - \cfrac{h_{i}}{6}\Delta m_{i-1} + m_{i-1}h_i + \cfrac{\Delta m_{i-1}h_i}{2}$

$S_{3,i}^{'}(x)|_{x=x_i} = \cfrac{1}{h_{i+1}}\Delta f_{i} - \cfrac{h_{i+1}}{2}m_{i} - \cfrac{h_{i+1}}{6}\Delta m_{i}$

$\cfrac{1}{h_{i}}\Delta f_{i-1} - \cfrac{h_{i}}{2}m_{i-1} - \cfrac{h_{i}}{6}\Delta m_{i-1} + m_{i-1}h_i + \cfrac{\Delta m_{i-1}h_i}{2} = \cfrac{1}{h_{i+1}}\Delta f_{i} - \cfrac{h_{i+1}}{2}m_{i} - \cfrac{h_{i+1}}{6}\Delta m_{i}$

$\cfrac{\Delta f_i}{h_{i+1}} - \cfrac{\Delta f_{i-1}}{h_{i}} = 
\cfrac{h_i}{6}m_{i-1} + \cfrac{h_i+h_{i+1}}{3}m_i + \cfrac{h_i}{6}m_{i+1}$

We assume that $h_{i+1} = h_i = const$, then

$$(5)\ \cfrac{\Delta f_i - \Delta f_{i-1}}{h} = 
\cfrac{h}{6}m_{i-1} + \cfrac{2h}{3}m_i + \cfrac{h}{6}m_{i+1},\ i = \overline{1, n-1}$$ <br>
$$\Uparrow$$ <br>
The dependence of the second derivatives ($m$'s) at the grid nodes on the increment of the function.

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Using (5), we obtain matix $A$

$A \cdot m = \delta \Rightarrow$
$\begin{bmatrix}
\cfrac{h}{6} &\cfrac{2h}{3} & \cfrac{h}{6}  & 0         & 0        & \dots & 0 & 0 & 0\\
0 & \cfrac{h}{6} & \cfrac{2h}{3} & \cfrac{h}{6}  & 0        & \dots & 0 & 0 & 0\\
0& 0 & \cfrac{h}{6}  & \cfrac{2h}{3} & \cfrac{h}{6} & \dots & 0 & 0 & 0\\
\dots & \dots & \dots & \dots & \dots & \dots & \dots & \dots & \dots     \\
0 & 0 & 0 & \dots & \dots & \dots & \cfrac{h}{6} & \cfrac{2h}{3} & \cfrac{h}{6}
\end{bmatrix}
\cdot
\begin{bmatrix}
m_0 \\
m_1 \\
m_2 \\
m_3 \\
\dots \\
m_n
\end{bmatrix}
=
\begin{bmatrix}
\cfrac{\Delta f_1 - \Delta f_0}{h} \\
\dots \\
\dots \\
\cfrac{\Delta f_{n-1} - \Delta f_{n-2}}{h} \\
\end{bmatrix}
;
$

Dimensionalities: 
$\underset{(n-1)\text{x}(n+1)}{A} \cdot \underset{(n+1)\text{x}1}{m} = \underset{(n-1)\text{x}1}{\delta}$

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

Unknown variables in this system are $m_i\ (i = \overline{0,n}) \Rightarrow$ we have $n+1$ unknown variables.

The system given in (5) is not closed with respect to an unknown variables, because we have $n - 1$. equations This happened, because we don't know function values at points $f_{-1}$ and $f_{n+1}$ in order to compute $m_0$ and $m_n$. Thus we need "border conditions". For natural spline these border conditions are: $m_0 = 0;\ m_n = 0$

Border conditions add 2 missing equations for the system (5) and make the whole system closed.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

Now we know that $m_0$ and $m_n$ are equal to $0 \Rightarrow$ remove first and last columns of matrix A (they do not influence calculations)<br>
Let's compose extended matrix including output vector (we have a system $Ax = b$, so add $b$ to A as last column).

Let last column values $\cfrac{\Delta f_i - \Delta f_{i-1}}{h}$ be $\delta_i$

And now we obtain extended matrix $A_1$, which is well-suited for tridiagonal matrix algorithm

$A_1 =
\begin{bmatrix}
\cfrac{2h}{3} & \cfrac{h}{6}  & 0         & 0        & \dots & 0 & 0 & \delta_1\\
\cfrac{h}{6}  & \cfrac{2h}{3} & \cfrac{h}{6}  & 0        & \dots & 0 & 0 & \delta_2\\
0         & \cfrac{h}{6}  & \cfrac{2h}{3} & \cfrac{h}{6} & \dots & 0 & 0 & \delta_3\\
\dots & \dots & \dots & \dots & \dots & \dots & \dots      \\
0 & 0 & \dots & \dots & \dots & \cfrac{h}{6} & \cfrac{2h}{3} & \delta_{n-1}
\end{bmatrix}
;\ m =
\begin{bmatrix}
m_1 \\
m_2 \\
m_3 \\
\dots \\
m_{n-1}
\end{bmatrix}
; m_0 = m_n = 0
$

$A_1(\text{without }\delta \text{ column}) \cdot m = \delta$ gives us $(n-1)$ equations, and $m_0 = m_n = 0$  2 more - in total $n+1$ equations

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Now we need to apply alogrithm of direct pass by Gauss and get enchanced matrix $\hat{A}_1$

$\hat{A}_1 =
\begin{bmatrix}
1 & \cfrac{1}{4}  & 0         & 0        & \dots & 0 & 0 & \cfrac{3\delta_1}{2h} (=Q_1)\\
0  & 1 & \cfrac{4}{15}  & 0        & \dots & 0 & 0 & \cfrac{24\cdot(\delta_2 - Q_1*\cfrac{h}{6})}{15h} (=Q_2)\\
0         & 0  & 1 & \cfrac{15}{56} & \dots & 0 & 0 & \cfrac{5(\delta_3-Q_2*\cfrac{h}{6})}{3h} (=Q_3)\\
\dots & \dots & \dots & \dots & \dots & \dots & \dots      \\
0 & 0 & \dots & \dots & \dots & 0 & 1 & {Q_{n-1}}\\
\end{bmatrix}
$

General forms of these matrices are

$
A_1 =
\begin{bmatrix}
b_1 & c_1 & 0 & 0 & \dots & 0 & 0 & \delta_1\\
a_2 & b_2 & c_2 & 0 & \dots & 0 & 0 & \delta_2\\
0 & a_3 & b_3 & c_3 & \dots & 0 & 0 & \delta_3\\
\dots & \dots & \dots & \dots & \dots & \dots & \dots      \\
0 & 0 & \dots & \dots & \dots & a_{n-1} & b_{n-1} & \delta_{n-1}\\
\end{bmatrix}
;\ \hat{A}_1 =
\begin{bmatrix}
1 & P_1 & 0 & 0 & \dots & 0 & 0 & Q_1\\
0 & 1 & P_2 & 0 & \dots & 0 & 0 & Q_2\\
0 & 0 & 1 & P_3 & \dots & 0 & 0 & Q_3\\
\dots & \dots & \dots & \dots & \dots & \dots & \dots      \\
0 & 0 & \dots & \dots & \dots & 0 & 1 & {Q_{n-1}}\\
\end{bmatrix}
$

First of all, we need to apply Gauss elimination for tridiagonal matrix, which is done by <br>
substituring $c_i$ with recursively computed values ($P_i$), $b_i$ with $1$ and $a_i$ with $0$<br>

To compute $P_i$ we divide $c_i$ by the row normalization element - make $b_i = 1$(row diagonal element($b_i$) - coefficient makes $a_i = 0$
($a_i\cdot P_{i-1}$)), so <br>

(6) $P_i = \cfrac{c_i}{b_i - a_i \cdot P_{i-1}} = \cfrac{\cfrac{h}{6}}{\cfrac{2h}{3}-\cfrac{h}{6}\cdot P_{i-1}} =
\cfrac{1}{4-P_{i-1}}$, if $i=1$ then compute without $P_{i-1}$ <br>

To compute $Q_i$ we subtract $a_i \cdot Q_{i-1}$, which corresponds to zeroing $a_i$ and then divide by the same value as $P_i$ <br>(corresponds to normalizing the row -> making $b_i = 1$), so <br>

(7) $Q_i = \cfrac{\delta_i - a_i*Q_{i-1}}{b_i - a_i \cdot P_{i-1}} =
\cfrac{6\delta_i-h Q_{i-1}}{h(4-P_{i-1})}$, if $i=1$ then compute without $hQ_{i-1}$ <br> 

**Formulas for $P_i \text{ and } Q_i$ correspond to direct pass (computing this coefficients to get $\hat{A}_1$ from $A_1$)**<br>

Reverse pass starts with computing $m_{n-1}$ as $Q_{n-1}$ (already computed during direct pass)<br>
Having a system with $P's$ and $Q's$ we can compute unknown variables ($m's$) as follows <br>
restructure system of linear equations back as<br>
$\hat{A}_1(\text{without } Q) \cdot m = Q$ <br>
$\Downarrow$

(8) $m_i = Q_i - m_{i+1}\cdot P_i $, $i = \overline{1, n-1}$, $m_0 = m_n = 0$

**Formula (8) corresponds to reverse pass of tridiagonal algorithm**

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [18]:
def compute_PQ(N: int, f: list, h: float) -> (list, list):
    """Compute P and Q coefficients, according to 
    formulas (6) and (7).
    """
    pass

def compute_m(Q: list, P: list)-> list:
    """Compute m coefficients, according to formula (8)."""
    pass

def compute_coefficients(f: list, m: list, h: float) -> (list, list):
    """Compute a coefficients, according to task 5 a1, a2, a3, a4 values."""
    pass

def compute_value(S: list, x: float, x_i: float) -> float:
    """Compute function value in point of spline"""
    dx = x - x_i
    return S[0] + S[1] * dx + S[2] * dx**2 + S[3] * dx**3

def main():
    read_values(N, x, f)
    h = x[1] - x[0] # if N > 1
    P, Q = compute_PQ(N, f, h)
    m = compute_m(Q, P)
    S[N - 1] # spline for N-1 pieces, 4 coef. each
    for i in range(N-1):
        a, b, c, d = compute_coefficients(f, m, h)
        S[i] = (a, b, c, d)
        
    read_values(x_new)
    for xd in x_new:
        if xd belongs to i-th cell:
            res = compute_value(S[i], xd, x_i)
            print(res)


### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

In "Численные методы в примерах и задачах" by V.I.Kireev and A.V.Panteleev, page 192, they say that the method error can be estimated by following formula

$||f^{(p)}(x) - S_3^{(p)}(x)||_{C[a,b]} = \max_{[a, b]}|f^(p)(x) - S_3^{(p)}(x)|
\le M_4h^{4-p}$, for $p=0,1,2$

If we consider natural spline, the artificial border conditions ($m_0 = m_n = 0$) can not be hold for some interpolated functions, which leads to the reduction of precision. Instead of 4'th order precision, natural spline has only 2'nd order precision according to [source](http://scask.ru/i_book_clm.php?id=90). <br>

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

To perform the algorithm we use 
- 1) direct pass - Gauss Elimination for tridiagonal matrix, which has complexity $O(n)$ source: https://algowiki-project.org/ru/Компактная_схема_метода_Гаусса_для_трёхдиагональной_матрицы,_последовательный_вариант
- 2) reverse pass - We use recurrent formula $x_i = P_ix_{i+1} + Q_i$, where $P_i$ and $Q_i$ are precomputed from original tridiagonal matrix<br>
This step has the same time complexity of $O(n)$

So, the total complexity is $O(n) + O(n) = O(n)$

### Congrats!